In [ ]:
!git clone https://github.com/Thehunk1206/Zero-DCE.git

Cloning into 'Zero-DCE'...
remote: Enumerating objects: 373, done.
remote: Counting objects: 100% (372/372), done.
remote: Compressing objects: 100% (210/210), done.
remote: Total 373 (delta 171), reused 356 (delta 158), pack-reused 1
Receiving objects: 100% (373/373), 59.22 MiB | 17.54 MiB/s, done.
Resolving deltas: 100% (171/171), done.


In [ ]:
%cd Zero-DCE
!mkdir dataset_v 
!mkdir dataset200
!mkdir validate

%cd validate
!mkdir low
!mkdir high
!mkdir high15
%cd /content/Zero-DCE

/content/Zero-DCE


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import tensorflow as tf
from tqdm import tqdm

In [ ]:
from skimage import metrics, transform
import cv2
import glob

In [ ]:
psnr=[]
ssim=[]
resize_imghigh=[]
resize_imglow=[]
print(os.listdir("validate/high15/"))
for img_h in os.listdir("validate/high15/"):
  #print(type(img_h))
  img_re=cv2.imread("validate/high15/"+img_h)
  res = cv2.resize(img_re, dsize=(512,512), interpolation=cv2.INTER_CUBIC)
  #print(res.shape)
  resize_imghigh.append(res)

for img_l in os.listdir("output/validate0.001/"):
  img_re=cv2.imread("output/validate0.001/"+img_l)
  res = cv2.resize(img_re, dsize=(512,512), interpolation=cv2.INTER_CUBIC)
  #print(res.shape)
  resize_imglow.append(res)

for img_true,img_test in zip(resize_imghigh,resize_imglow):
  psnr.append(metrics.peak_signal_noise_ratio(img_true,img_test))
  ssim.append(metrics.structural_similarity(img_true,img_test,multichannel=True))


['748.png', '669.png', '146.png', '23.png', '111.png', '493.png', '79.png', '778.png', '179.png', '22.png', '1.png', '665.png', '547.png', '55.png', '780.png']


In [ ]:
"""Parameters
      Dataset_size=56
      Epoch=40
      learning rate=1e-3
      filters=32
        """

!python train_model.py --dataset_dir dataset_v/ \
                       --model_type zero_dce \
                       --checkpoint_dir Trained_model/ \
                       --IMG_H 512 \
                       --IMG_W 512 \
                       --epoch 40\
                       --batch_size 3 \
                       --learning_rate 0.001 \
                       --filters 32 \
                       --iteration 8

Creating dataset pipeline...

Creating Zero-DCE model...

Compiling the model...

[INFO] Summary of model

Model: "DCE-net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 512, 512, 32  9248        ['conv2d[0][0]']   

In [ ]:
"""Parameters 
removed l_dcp from total_loss
      Dataset_size=56
      Epoch=40
      learning rate=1e-3
      filters=32
        """

!python train_model.py --dataset_dir dataset_v/ \
                       --model_type zero_dce \
                       --checkpoint_dir Trained_model/ \
                       --IMG_H 512 \
                       --IMG_W 512 \
                       --epoch 40\
                       --batch_size 4 \
                       --learning_rate 0.001 \
                       --filters 32 \
                       --iteration 8

Creating dataset pipeline...

Creating Zero-DCE model...

Compiling the model...

[INFO] Summary of model

Model: "DCE-net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 512, 512, 32  9248        ['conv2d[0][0]']   

#calculating PSNR and SSIM

In [ ]:
import pandas as pd
metrics_df=pd.DataFrame(list(zip(psnr,ssim)),columns=['PSNR','SSIM'])

In [ ]:
#data size of 56 with  DCP addded in input layer
average_psnr=sum(psnr)/len(psnr)
print("Average PSNR:",round(average_psnr,2))

average_ssim=sum(ssim)/len(ssim)
print("Average SSIM:",round(average_ssim,2))

Average PSNR: 11.22
Average SSIM: 0.33


In [ ]:
#data size of 56 with no DCP addded in input layer
average_psnr=sum(psnr)/len(psnr)
print("Average PSNR:",round(average_psnr,2))

average_ssim=sum(ssim)/len(ssim)
print("Average SSIM:",round(average_ssim,2))

Average PSNR: 11.22
Average SSIM: 0.29


In [ ]:
#data size of 56 with dcp multiplied in enhanced image 
  #le_img = le_img + a_map * (tf.square(le_img) - le_img)*output4D
average_psnr=sum(psnr)/len(psnr)
print("Average PSNR:",round(average_psnr,2))

average_ssim=sum(ssim)/len(ssim)
print("Average SSIM:",round(average_ssim,2))

Average PSNR: 5.73
Average SSIM: 0.27


# Addding a DCP loss function 

In [ ]:
#data size of 56 with dcp added as loss-> (w=10)*(1-dcp)
average_psnr=sum(psnr)/len(psnr)
print("Average PSNR:",round(average_psnr,2))

average_ssim=sum(ssim)/len(ssim)
print("Average SSIM:",round(average_ssim,2))

Average PSNR: 5.5
Average SSIM: 0.29


In [ ]:
metrics_df

,PSNR,SSIM
0,4.189640,0.353446
1,7.475114,0.188505
2,6.503444,0.201763
3,6.113444,0.386447
4,8.165824,0.168955
5,3.261740,0.141069
6,7.825957,0.491394
7,3.309498,0.206870
8,3.059240,0.304414
9,7.220041,0.483501


In [ ]:
#data size of 56 with dcp added as loss-> (w=1)*(1-dcp)
average_psnr=sum(psnr)/len(psnr)
print("Average PSNR:",round(average_psnr,2))

average_ssim=sum(ssim)/len(ssim)
print("Average SSIM:",round(average_ssim,2))

Average PSNR: 5.5
Average SSIM: 0.29


In [ ]:
metrics_df

,PSNR,SSIM
0,4.189640,0.353446
1,7.475114,0.188505
2,6.503444,0.201763
3,6.113444,0.386447
4,8.165824,0.168955
5,3.261740,0.141069
6,7.825957,0.491394
7,3.309498,0.206870
8,3.059240,0.304414
9,7.220041,0.483501


In [ ]:
#data size of 56 with dcp added as loss-> (w=0.1)*(1-dcp)
average_psnr=sum(psnr)/len(psnr)
print("Average PSNR:",round(average_psnr,2))

average_ssim=sum(ssim)/len(ssim)
print("Average SSIM:",round(average_ssim,2))

Average PSNR: 5.5
Average SSIM: 0.29


In [ ]:
metrics_df

,PSNR,SSIM
0,4.189640,0.353446
1,7.475114,0.188505
2,6.503444,0.201763
3,6.113444,0.386447
4,8.165824,0.168955
5,3.261740,0.141069
6,7.825957,0.491394
7,3.309498,0.206870
8,3.059240,0.304414
9,7.220041,0.483501


In [ ]:
#data size of 56 with dcp added as loss-> (w=200)*(1-dcp)
average_psnr=sum(psnr)/len(psnr)
print("Average PSNR:",round(average_psnr,2))

average_ssim=sum(ssim)/len(ssim)
print("Average SSIM:",round(average_ssim,2))

Average PSNR: 5.5
Average SSIM: 0.29


In [ ]:
metrics_df

In [ ]:
#data size of 56 with dcp added as loss-> (w=0.001)*(1-dcp)
average_psnr=sum(psnr)/len(psnr)
print("Average PSNR:",round(average_psnr,2))

average_ssim=sum(ssim)/len(ssim)
print("Average SSIM:",round(average_ssim,2))

Average PSNR: 5.5
Average SSIM: 0.29


In [ ]:
#data size of 56 with dcp added as loss-> l_dcp removed
average_psnr=sum(psnr)/len(psnr)
print("Average PSNR:",round(average_psnr,2))

average_ssim=sum(ssim)/len(ssim)
print("Average SSIM:",round(average_ssim,2))

Average PSNR: 11.37
Average SSIM: 0.32


In [ ]:
"""Parameters
 weight of DCP=1
      Dataset_size=56
      Epoch=40
      learning rate=1e-3
      filters=32
        """

!python train_model.py --dataset_dir dataset_v/ \
                       --model_type zero_dce \
                       --checkpoint_dir Trained_model/ \
                       --IMG_H 512 \
                       --IMG_W 512 \
                       --epoch 40\
                       --batch_size 4 \
                       --learning_rate 0.001 \
                       --filters 32 \
                       --iteration 8

Creating dataset pipeline...

Creating Zero-DCE model...

Compiling the model...

[INFO] Summary of model

Model: "DCE-net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 512, 512, 32  9248        ['conv2d[0][0]']   

In [ ]:
psnr=[]
ssim=[]
resize_imghigh=[]
resize_imglow=[]
print(os.listdir("validate/high15/"))
for img_h in os.listdir("validate/high15/"):
  #print(type(img_h))
  img_re=cv2.imread("validate/high15/"+img_h)
  res = cv2.resize(img_re, dsize=(512,512), interpolation=cv2.INTER_CUBIC)
  #print(res.shape)
  resize_imghigh.append(res)

for img_l in os.listdir("output/validate/"):
  img_re=cv2.imread("output/validate/"+img_l)
  res = cv2.resize(img_re, dsize=(512,512), interpolation=cv2.INTER_CUBIC)
  #print(res.shape)
  resize_imglow.append(res)

for img_true,img_test in zip(resize_imghigh,resize_imglow):
  psnr.append(metrics.peak_signal_noise_ratio(img_true,img_test))
  ssim.append(metrics.structural_similarity(img_true,img_test,multichannel=True))

['111.png', '780.png', '778.png', '79.png', '22.png', '493.png', '669.png', '23.png', '547.png', '179.png', '55.png', '665.png', '146.png', '1.png', '748.png']


In [ ]:
import pandas as pd
metrics_df=pd.DataFrame(list(zip(psnr,ssim)),columns=['PSNR','SSIM'])

In [ ]:
metrics_df

,PSNR,SSIM
0,7.047169,0.548463
1,2.621058,0.244541
2,4.939814,0.071449
3,8.041169,0.342015
4,5.625899,0.382091
5,2.815424,0.180565
6,6.268335,0.398966
7,6.816985,0.253473
8,5.156609,0.188595
9,3.052501,0.222098


In [ ]:
#data size of 56 with dcp added as loss weight=1
average_psnr=sum(psnr)/len(psnr)
print("Average PSNR:",round(average_psnr,2))

average_ssim=sum(ssim)/len(ssim)
print("Average SSIM:",round(average_ssim,2))

Average PSNR: 5.45
Average SSIM: 0.29


In [ ]:
! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/1.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8

[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7f24cda333d0>
[info] running inference....
[info] inference time: 3212.358236312866 ms
[info] saved enhanced image to disk


In [ ]:
! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/111.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8

[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7ff224e15b80>
[info] running inference....
[info] inference time: 2509.2601776123047 ms
[info] saved enhanced image to disk


In [ ]:
! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/146.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8

[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7f1674c838e0>
[info] running inference....
[info] inference time: 2476.3569831848145 ms
[info] saved enhanced image to disk


In [ ]:
! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/179.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8

[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7f37f2d51f10>
[info] running inference....
[info] inference time: 2446.7880725860596 ms
[info] saved enhanced image to disk


In [ ]:

! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/22.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8

[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7fc25cd74ee0>
[info] running inference....
[info] inference time: 2509.8559856414795 ms
[info] saved enhanced image to disk


In [ ]:

! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/23.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8

[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7f82a71d0d00>
[info] running inference....
[info] inference time: 2468.4078693389893 ms
[info] saved enhanced image to disk


In [ ]:
! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/493.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8

[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7fc4cc5b0ee0>
[info] running inference....
[info] inference time: 2460.808515548706 ms
[info] saved enhanced image to disk


In [ ]:
! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/547.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8


[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7f5eddffdaf0>
[info] running inference....
[info] inference time: 2493.603229522705 ms
[info] saved enhanced image to disk


In [ ]:
! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/55.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8

[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7f290174fbb0>
[info] running inference....
[info] inference time: 2495.128870010376 ms
[info] saved enhanced image to disk


In [ ]:
! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/665.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8

[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7ff2cfda8e50>
[info] running inference....
[info] inference time: 2488.345146179199 ms
[info] saved enhanced image to disk


In [ ]:
! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/669.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8

[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7f63dd16af40>
[info] running inference....
[info] inference time: 2505.733013153076 ms
[info] saved enhanced image to disk


In [ ]:
! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/748.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8

[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7f70d08d6280>
[info] running inference....
[info] inference time: 2482.578754425049 ms
[info] saved enhanced image to disk


In [ ]:
! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/778.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8

[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7ff42b980400>
[info] running inference....
[info] inference time: 2502.7811527252197 ms
[info] saved enhanced image to disk


In [ ]:
! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/780.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8

[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7f6035a82f40>
[info] running inference....
[info] inference time: 2468.111515045166 ms
[info] saved enhanced image to disk


In [ ]:
! python command_line_scripts/single_image_enhance.py --model_path Trained_model/zero_dce_iter8/zero_dce_512x512_iter8_40 \
                                 --img_h 512 \
                                 --img_w 512 \
                                 --image_path validate/low/79.png \
                                 --plot 0 \
                                 --save_result 1 \
                                 --iteration 8

[info] loading model from disk....
loaded model: <keras.saving.saved_model.load.ZeroDCE object at 0x7f23d5cedd60>
[info] running inference....
[info] inference time: 2469.351291656494 ms
[info] saved enhanced image to disk


In [ ]:
def dark_channel(img,size):
  img=tf.math.reduce_min(img,axis=-1) #finds min and convert img to img_hximg_w
  
  img=tf.stack([img,img,img],axis=-1) #converting to 3 dimensional
  
  img=tf.expand_dims(img,axis=0) # converting to 4 dimensional

  kernel=tf.zeros((size,size,3),tf.float32)
  output4D = tf.nn.erosion2d(img, filters=kernel, strides=(1,1,1,1),data_format='NHWC', dilations=(1,1,1,1),padding="SAME")
  #output4D = output4D - tf.ones_like(output4D)
  return output4D

In [ ]:
img_raw=tf.io.read_file("23.png")
img=tf.image.decode_png(img_raw,channels=3)
img=tf.image.convert_image_dtype(img,tf.float32)
print(tf.reduce_min(img[0:7,0:7,0]))

tf.Tensor(0.20784315, shape=(), dtype=float32)


In [ ]:
dcp=dark_channel(img,15)
print(dcp.shape)
print(dcp[:,0:5,0:5,:])

(1, 400, 600, 3)
tf.Tensor(
[[[[0.20784315 0.18431373 0.14901961]
   [0.20784315 0.18431373 0.14901961]
   [0.20784315 0.18431373 0.14901961]
   [0.20784315 0.18431373 0.14901961]
   [0.20784315 0.18431373 0.14901961]]

  [[0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]]

  [[0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]]

  [[0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]]

  [[0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]
   [0.20784315 0.18039216 0.14901961]]]], shape=(1, 

In [ ]:
tf.ones([4,512,512,3])*tf.zeros([4,512,512,1])

<tf.Tensor: shape=(4, 512, 512, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


  

In [ ]:
"""Parameters
      Dataset_size=56
      Epoch=40
      learning rate=1e-3
      filters=32
        """

!python train_model.py --dataset_dir dataset_v/ \
                       --model_type zero_dce \
                       --checkpoint_dir Trained_model/ \
                       --IMG_H 512 \
                       --IMG_W 512 \
                       --epoch 40\
                       --batch_size 3 \
                       --learning_rate 0.001 \
                       --filters 32 \
                       --iteration 8

Creating dataset pipeline...

Creating Zero-DCE model...

Compiling the model...

[INFO] Summary of model

Model: "DCE-net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 512, 512, 32  9248        ['conv2d[0][0]']   